Option chains
=======

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('20.230.202.72', 7497, clientId=550057)

<IB connected to 20.230.202.72:7497 clientId=550057>

Suppose we want to find the options on the SPX, with the following conditions:

* Use the next three monthly expiries;
* Use strike prices within +- 20 dollar of the current SPX value;
* Use strike prices that are a multitude of 5 dollar.

To get the current market value, first create a contract for the underlyer (the S&P 500 index):

In [2]:
# spx = Index('SPX', 'CBOE')
contract = Future(symbol="NQ", localSymbol="NQU3", exchange="CME")
ib.qualifyContracts(contract)

[Future(conId=568549928, symbol='NQ', lastTradeDateOrContractMonth='20230915', multiplier='20', exchange='CME', currency='USD', localSymbol='NQU3', tradingClass='NQ')]

To avoid issues with market data permissions, we'll use delayed data:

In [3]:
ib.reqMarketDataType(1)

Then get the ticker. Requesting a ticker can take up to 11 seconds.

In [4]:
[ticker] = ib.reqTickers(contract)
ticker

Ticker(contract=Future(conId=568549928, symbol='NQ', lastTradeDateOrContractMonth='20230915', multiplier='20', exchange='CME', currency='USD', localSymbol='NQU3', tradingClass='NQ'), time=datetime.datetime(2023, 7, 19, 3, 40, 46, 898203, tzinfo=datetime.timezone.utc), minTick=0.25, bid=15959.75, bidSize=1.0, ask=15960.0, askSize=5.0, last=15959.75, lastSize=4.0, volume=15278.0, open=15949.25, high=15967.75, low=15945.0, close=15974.75, halted=0.0, ticks=[TickData(time=datetime.datetime(2023, 7, 19, 3, 40, 46, 898203, tzinfo=datetime.timezone.utc), tickType=4, price=15959.75, size=4.0), TickData(time=datetime.datetime(2023, 7, 19, 3, 40, 46, 898203, tzinfo=datetime.timezone.utc), tickType=49, price=0.0, size=0), TickData(time=datetime.datetime(2023, 7, 19, 3, 40, 46, 898203, tzinfo=datetime.timezone.utc), tickType=8, price=-1.0, size=15278.0), TickData(time=datetime.datetime(2023, 7, 19, 3, 40, 46, 898203, tzinfo=datetime.timezone.utc), tickType=6, price=15967.75, size=0.0), TickData(ti

Take the current market value of the ticker:

In [5]:
spxValue = ticker.marketPrice()
spxValue

15959.75

The following request fetches a list of option chains:

In [10]:
ib.reqContractDetails(contract)

[ContractDetails(contract=Contract(secType='FUT', conId=568549928, symbol='NQ', lastTradeDateOrContractMonth='20230915', multiplier='20', exchange='CME', currency='USD', localSymbol='NQU3', tradingClass='NQ'), marketName='NQ', minTick=0.25, orderTypes='ACTIVETIM,AD,ADJUST,ALERT,ALGO,ALLOC,AVGCOST,BASKET,BENCHPX,COND,CONDORDER,DAY,DEACT,DEACTDIS,DEACTEOD,GAT,GTC,GTD,GTT,HID,ICE,IOC,LIT,LMT,LTH,MIT,MKT,MTL,NGCOMB,NONALGO,OCA,PEGBENCH,SCALE,SCALERST,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF', validExchanges='CME,QBALGO', priceMagnifier=1, underConId=11004958, longName='E-mini NASDAQ 100 ', contractMonth='202309', industry='', category='', subcategory='', timeZoneId='US/Central', tradingHours='20230717:1700-20230718:1600;20230718:1700-20230719:1600;20230719:1700-20230720:1600;20230720:1700-20230721:1600;20230722:CLOSED;20230723:1700-20230724:1600', liquidHours='20230718:0830-20230718:1600;20230719:0830-20230719:1600;20230720:0830-20230720:1600;20230721:0830

In [7]:
chains = ib.reqSecDefOptParams(contract.symbol, '', contract.secType, contract.conId)

util.df(chains)

Error 321, reqId 5: Error validating request.-'cw' : cause - Missing exchange for security type FUT


These are four option chains that differ in ``exchange`` and ``tradingClass``. The latter is 'SPX' for the monthly and  'SPXW' for the weekly options. Note that the weekly expiries are disjoint from the monthly ones, so when interested in the weekly options the monthly options can be added as well.

In this case we're only interested in the monthly options trading on SMART:

In [7]:
chain = next(c for c in chains if c.tradingClass == 'SPX' and c.exchange == 'SMART')
chain

OptionChain(exchange='SMART', underlyingConId='416904', tradingClass='SPX', multiplier='100', expirations=['20230720', '20230817', '20230914', '20231019', '20231116', '20231214', '20240118', '20240215', '20240314', '20240418', '20240516', '20240620', '20240718', '20240919', '20241219', '20250618', '20251218', '20261217', '20271216', '20281214'], strikes=[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700.0, 800.0, 900.0, 1000.0, 1100.0, 1200.0, 1300.0, 1400.0, 1500.0, 1600.0, 1700.0, 1750.0, 1800.0, 1850.0, 1900.0, 1950.0, 2000.0, 2050.0, 2100.0, 2150.0, 2200.0, 2250.0, 2300.0, 2350.0, 2400.0, 2450.0, 2500.0, 2525.0, 2550.0, 2575.0, 2600.0, 2625.0, 2650.0, 2675.0, 2700.0, 2725.0, 2750.0, 2775.0, 2800.0, 2825.0, 2850.0, 2875.0, 2900.0, 2925.0, 2950.0, 2975.0, 3000.0, 3025.0, 3050.0, 3075.0, 3100.0, 3125.0, 3150.0, 3175.0, 3200.0, 3225.0, 3250.0, 3275.0, 3280.0, 3290.0, 3300.0, 3310.0, 3320.0, 3325.0, 3330.0, 3340.0, 3350.0, 3360.0, 3370.0, 3375.0, 3380.0, 3390.0, 3400.0, 3410.0, 3420.0, 3425

What we have here is the full matrix of expirations x strikes. From this we can build all the option contracts that meet our conditions:

In [8]:
strikes = [strike for strike in chain.strikes
        if strike % 5 == 0
        and spxValue - 20 < strike < spxValue + 20]
expirations = sorted(exp for exp in chain.expirations)[:3]
rights = ['P', 'C']

contracts = [Option('SPX', expiration, strike, right, 'SMART', tradingClass='SPX')
        for right in rights
        for expiration in expirations
        for strike in strikes]

contracts = ib.qualifyContracts(*contracts)
len(contracts)

48

In [10]:
contracts[0]

Option(conId=611886528, symbol='SPX', lastTradeDateOrContractMonth='20230720', strike=4410.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   230721P04410000', tradingClass='SPX')

Now to get the market data for all options in one go:

In [11]:
tickers = ib.reqTickers(*contracts)


Ticker(contract=Option(conId=611886528, symbol='SPX', lastTradeDateOrContractMonth='20230720', strike=4410.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   230721P04410000', tradingClass='SPX'), time=datetime.datetime(2023, 7, 7, 19, 1, 18, 274862, tzinfo=datetime.timezone.utc), minTick=0.05, bid=25.5, bidSize=225.0, bidExchange='C', ask=25.9, askSize=225.0, askExchange='C', last=24.3, lastSize=20.0, volume=116.0, high=36.85, low=22.8, close=36.15, halted=0.0, bidGreeks=OptionComputation(tickAttrib=0, impliedVol=0.10504980433330299, delta=-0.40226895081485237, optPrice=25.600000381469727, pvDividend=2.113739206668964, gamma=0.00423973046565308, vega=3.361150592543191, theta=-0.9939242989485626, undPrice=4424.78), askGreeks=OptionComputation(tickAttrib=0, impliedVol=0.10626041702911247, delta=-0.4032686807629613, optPrice=26.100000381469727, pvDividend=2.113739206668964, gamma=0.004194093946703964, vega=3.363288680496201, theta=-1.0084836488424833, un

In [12]:

tickers[0]

Ticker(contract=Option(conId=611886528, symbol='SPX', lastTradeDateOrContractMonth='20230720', strike=4410.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   230721P04410000', tradingClass='SPX'), time=datetime.datetime(2023, 7, 7, 19, 1, 18, 274862, tzinfo=datetime.timezone.utc), minTick=0.05, bid=25.5, bidSize=225.0, bidExchange='C', ask=25.9, askSize=225.0, askExchange='C', last=24.3, lastSize=20.0, volume=116.0, high=36.85, low=22.8, close=36.15, halted=0.0, bidGreeks=OptionComputation(tickAttrib=0, impliedVol=0.10504980433330299, delta=-0.40226895081485237, optPrice=25.600000381469727, pvDividend=2.113739206668964, gamma=0.00423973046565308, vega=3.361150592543191, theta=-0.9939242989485626, undPrice=4424.78), askGreeks=OptionComputation(tickAttrib=0, impliedVol=0.10626041702911247, delta=-0.4032686807629613, optPrice=26.100000381469727, pvDividend=2.113739206668964, gamma=0.004194093946703964, vega=3.363288680496201, theta=-1.0084836488424833, un

Peer closed connection.


The option greeks are available from the ``modelGreeks`` attribute, and if there is a bid, ask resp. last price available also from ``bidGreeks``, ``askGreeks`` and ``lastGreeks``. For streaming ticks the greek values will be kept up to date to the current market situation.

In [11]:
ib.disconnect()